___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://timelyportfolio.github.io/rCharts_time_series/assets/fig/unnamed-chunk-14.png" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2024
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://timelyportfolio.github.io/rCharts_time_series/assets/fig/unnamed-chunk-14.png</p>

### <font color= #2E9AFE> Tema: Series de Tiempo con Clustering</font>

#### Ejemplo: Diversificación de Portafolios

<img style="float: center; margin: 0px 0px 15px 15px;" src="https://moneymorning.com/wp-content/blogs.dir/1/files/2021/04/DIVERS1-1024x573.png" width="350px" height="180px" />

Un portafolio de inversión de activos financieros (o cartera de inversión) es la forma de agrupar diversos tipos de activos, a fin de invertir un patrimonio y alcanzar, en algún plazo definido, un objetivo de rentabilidad dada ciertas consideraciones de riesgo.

Lo que se quiere hacer es idear una estrategia en la que seleccione sólo aquellas acciones que se comporten de manera diferente. Esto ayudará a mitigar el riesgo y una forma de hacerlo es seleccionar acciones de diferentes sectores, pero una solución más basada en datos puede ser aplicar el algoritmo de agrupación K-Means en datos bursátiles para identificar diferentes grupos de acciones.

Este enfoque permite a los inversores construir portafolios diversificados basados en los grupos formados. Por ejemplo, se podrían identificar acciones con bajo riesgo y alto rendimiento en un clúster y otras con mayores fluctuaciones en otro.


**Objetivo**: aprender sobre acciones de la bolsa utilizando algoritmos de clustering para agrupar acciones que tienen un comportamiento similar. 

Vamos a enfocarnos en acciones que son parte del índice del Dow Jones. También en algunas del índice del S&P500

Tanto el Dow Jones, como el S&P 500 son índices bursátiles del mercado estadounidense.

Estos índices se calculan a partir de la cotizaciones de las empresas que se agrupan en ellos. 

**Dow Jones**

El Dow Jones aglutina las 30 mayores empresas cotizadas de Estados Unidos proveniente de los distintos sectores de la Industria excepto de transporte y energía que se miden en índices separados.

**S&P 500**

Se suelte considerar por los analistas como el indicador más próximo a la situación real en la que se encuentra el mercado.

Bajo este índice se agrupan las 500 mayores compañías estadounidenses que cotizan en bolsa, concretamente en la bolsa de Nueva York.

Estas compañías no son fijas, son seleccionadas por un comité de la firma S&P, y deben cumplir una serie de requisitos para poder ser incluidas en dicho índice, relacionados con su capitalización bursátil, liquidez, capital flotante, viabilidad financiera, antigüedad en bolsa y su negociación en la bolsa de valores.

### Datos

Vamos a utilizar varias fuentes de datos:
- Lista de compañías que son parte del índice del Dow Jones: https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average
- Datos de los últimos 5 años del índice del S&P500
- Alguna información sobre los componentes de las acciones del S&P500, principalmente el símbolo de cada compañía

#### S&P500

Se tienen datos de acciones de cada compañía que es parte del índice, etiquetadas por su nombre de cotización en la bolsa. 

- Date: en formato yy-mm-dd
- Open: precio de la acción cuando el mercado abre (USD)
- Low: precio más bajo alcanzado durante el día
- Close: precio al que se cierra en el día
- High: precio más alto alcanzado durante el día
- Volume: número de acciones negociadas
- Name: nombre de la acción en la bolsa


In [ ]:
pip install yfinance

In [2]:
#librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import yfinance as yf
from datetime import datetime, timedelta

random.seed(42)
plt.rcParams['figure.figsize'] = (7,4.5) 

In [3]:
# Obtener la lista de empresas del S&P 500, (se puede descargar de wikipedia)
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)
    sp500_table = table[0]
    tickers = sp500_table['Symbol'].tolist()
    return tickers

start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

def download_prices(tickers, start_date, end_date):
    print("Descargando datos de cierre ajustado para el S&P 500...")
    data = yf.download(tickers, start=start_date, end=end_date, group_by="ticker", progress=True)
    return data

tickers = get_sp500_tickers()

data = download_prices(tickers, start_date, end_date)

data_close = data.loc[:, (slice(None), 'Adj Close')]  # Seleccionar solo los precios de cierre ajustados
data_close.columns = [col[0] for col in data_close.columns] 
data_close.reset_index(inplace=True)

data_close.to_excel("sp500_prices.xlsx", index=False)


Descargando datos de cierre ajustado para el S&P 500...


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2019-11-24 -> 2024-11-22)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
C:\Users\vivia\AppData\Local\Temp\ipykernel_24636\598747937.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_close.reset_index(inplace=True)


In [4]:
#Cargar precios de las acciones del S&P500
data = pd.read_excel('../../data/raw/sp500_prices.xlsx')
data = data.rename(columns={'Name':'symbol'})
data['Date'] = pd.to_datetime(data['Date'])
data.head()

,Date,CTLT,AVGO,CSX,ALL,MMM,OXY,ROP,CLX,ORCL,...,TMUS,MTCH,CI,DOV,BRK.B,ELV,NVDA,J,MCK,NTRS
0,2019-11-22,52.040001,27.136105,22.031929,97.102745,114.490341,36.336483,345.880280,126.742714,52.202274,...,77.388893,69.870003,189.209915,101.769051,NaN,274.622009,5.246731,74.780922,146.966034,91.401436
1,2019-11-25,52.849998,27.541090,22.431883,97.632179,116.293793,36.657810,349.710938,125.867493,52.359653,...,76.807922,69.279999,191.103439,104.065102,NaN,276.560547,5.503481,72.674759,147.601608,91.682968
2,2019-11-26,52.669998,27.252430,22.481874,99.379410,115.603836,35.409252,352.870728,128.415176,52.313358,...,77.251030,70.370003,187.822617,104.557762,NaN,270.302673,5.398741,73.251350,140.407867,91.068665
3,2019-11-27,53.220001,27.439411,22.538313,99.290779,116.546539,35.574501,352.734650,129.073746,52.405945,...,77.772949,70.150002,187.503876,104.660469,NaN,272.382385,5.433597,74.100227,140.879730,91.879204
4,2019-11-29,51.990002,27.247259,22.428581,98.696915,115.972702,35.409252,350.372040,128.449799,51.970840,...,77.349518,70.480003,187.400742,104.081619,NaN,271.639008,5.396250,73.747879,139.682266,91.495277


In [6]:
#Quitar esas columnas que tienen todas las filas con datos nulos
data = data.dropna(axis=1, how='all')

In [ ]:
#Observamos el porcentaje de nulos que tienen las acciones
data[data.columns[data.isnull().any()]].isnull().sum() * 100 / data.shape[0]

In [ ]:
#Vamos a quitar esas acciones que tienen gran porcentaje de datos nulos
dropping = data[data.columns[data.isnull().any()]].isnull().sum() * 100 / data.shape[0]
dropping[dropping>1].index
data = data.drop(columns=['SW', 'AMTM', 'VLTO', 'CARR', 'PLTR', 'SOLV', 'KVUE', 'CEG', 'OTIS', 'ABNB', 'GEHC', 'GEV'], axis=1)

In [ ]:
#Quitamos los valores nulos
data = data.dropna()
data[data.columns[data.isnull().any()]].isnull().sum() * 100 / data.shape[0]

#### Lista de las industrias a las que pertenecen las compañías del S&P500 

Las descargamos de aquí:
https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [ ]:
def get_sp500_companies():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    tables = pd.read_html(url)
    
    sp500_table = tables[0]
    
    sp500_companies = sp500_table[["Symbol", "Security", "GICS Sector", "GICS Sub-Industry"]]
    
    sp500_companies.columns = ["Symbol", "Security", "Sector", "Sub-Industry"]
    
    return sp500_companies

companies = get_sp500_companies()

companies.to_csv("sp500_companies.csv", index=False)


In [ ]:
companies = companies.set_index(['Symbol'])
companies.head()

#### Lista de las compañías del Dow Jones

Las podemos descargar de aquí:
https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average

In [ ]:
import pandas as pd

def get_dow_jones_companies():
    url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
    
    tables = pd.read_html(url)
    
    dow_table = tables[2]
    
    dow_companies = dow_table[["Symbol", "Company"]]
    
    dow_companies.columns = ["Symbol", "Company"]
    
    return dow_companies

dow_data = get_dow_jones_companies()

dow_data.to_csv("dow_jones_companies.csv", index=False)


In [ ]:
dow_data.set_index("Symbol", inplace=True)
dow_data.head()

#### Preparar datos del S&P

In [ ]:
#Transformando los datos del S&P500 a serie de tiempo
acciones = data.set_index("Date")
acciones

In [ ]:
#Hacer que los datos sean semanales en lugar de diarios
acciones = acciones.resample('W').last() #se queda con el últimpo precio de cada semana
acciones.head()

#### Análisis Exploratorio

In [ ]:
#Función para extraer el símbolo de la acción
def get_name(symbol):
    name = symbol
    try:
        name = companies.loc[symbol]['security']
    except KeyError:
        pass
    return name

#Función para graficar la acción
def plot_stock(symbol, stocks=acciones):
    name = get_name(symbol)
    stocks[symbol].plot(title=name, label=name, alpha=0.9);

In [ ]:
plot_stock('MSFT')
plot_stock('AAPL')
plt.title('')
plt.legend()

#### Calcular los rendimientos para poder comparar las acciones

**Qué es un rendimiento?**

El rendimiento es uno de los principales indicadores de las inversiones en acciones, mediante el cual se puede evaluar su rentabilidad, viabilidad y compararlas entre sí. 

El rendimiento se calcula mediante la fórmula:

$$\frac{(\text{precio de venta - precio de compra}) }{ \text{precio de compra}} * 100\%$$

In [ ]:
#Rendimientos
start = acciones.iloc[0]
rendimientos = (acciones - start) / start

In [ ]:
rendimientos

In [ ]:
#Graficar rendimientos
plt.figure(figsize = (20,8))
plot_stock('MSFT', stocks=rendimientos)
plot_stock('AAPL', stocks=rendimientos)
plt.title('')
plt.legend();

#### ¿Cuáles son las mejores y peores acciones de los últimos 5 años?

In [ ]:
mejores = rendimientos.iloc[-1].sort_values(ascending=False).head()
peores = rendimientos.iloc[-1].sort_values().head()

In [ ]:
print('Mejores acciones')
names = pd.DataFrame({'name':[get_name(symbol) for symbol in mejores.index.tolist()]}, index = mejores.index)
mejores = pd.concat((mejores, names), axis=1)
mejores

In [ ]:
plot_stock('NVDA')

In [ ]:
plot_stock('NVDA', stocks=rendimientos)
plt.title('Rendimientos de Nvidia');

In [ ]:
print('Peores acciones')
names = pd.DataFrame({'name':[get_name(symbol) for symbol in peores.index.tolist()]}, index = peores.index)
peores = pd.concat((peores, names), axis=1)
peores

In [ ]:
plot_stock('WBA', stocks=rendimientos)
plt.title('Rendimientos de WBA');

#### Enfocándonos en las acciones del Dow Jones

In [ ]:
dow_acciones = [s for s in dow_data.index if s in acciones.columns]
dow_rendimientos = rendimientos[dow_acciones]

In [ ]:
print(dow_acciones)

In [ ]:
mejores_dow = dow_rendimientos.iloc[-1].sort_values(ascending=False).head()
peores_dow = dow_rendimientos.iloc[-1].sort_values().head()
print('Mejores acciones del Dow Jones')
names = pd.DataFrame({'name':[get_name(symbol) for symbol in mejores_dow.index.tolist()]}, index = mejores_dow.index)
mejores_dow = pd.concat((mejores_dow, names), axis=1)
mejores_dow

In [ ]:
print('Peores acciones del Dow Jones')
names = pd.DataFrame({'name':[get_name(symbol) for symbol in peores_dow.index.tolist()]}, index = peores_dow.index)
peores_dow = pd.concat((peores_dow, names), axis=1)
peores_dow

In [ ]:
plot_stock('BA', stocks=rendimientos);

#### Clustering

In [ ]:
dow_rendimientos.head()

In [ ]:
dow_rendimientos.shape

In [ ]:
#gráfica de codo para encontrar el k optimo
inertias = []
for k in range(2, 15):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(dow_rendimientos.T)
    inertias.append(kmeans.inertia_)
plt.plot(range(2,15), inertias)
plt.title('Inertia with dow_returns components');

In [ ]:
#pip install kneed

In [ ]:
from kneed import KneeLocator
kl = KneeLocator(
     range(2, 15), inertias, curve="convex", direction="decreasing")

kl.elbow

In [ ]:
#Aplicar el kmeans
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(dow_rendimientos.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

#Asignar cada dato a un cluster
for i,l in enumerate(kmeans.predict(dow_rendimientos.T)):
    clusters[l].append(dow_rendimientos.columns[i])

    #Que imprima los clusters    
for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(get_name(symbol) + ' (' + symbol + ')', end=' ### ')
    print()
    print()

In [ ]:
#Graficar todos los clusters
for c in sorted(clusters):
    plt.figure(figsize = (20,8))
    for symbol in clusters[c]:
        plot_stock(symbol, stocks=dow_rendimientos)
    plt.title('Cluster de rendimientos' + str(c))
    plt.legend()
    plt.show()